In [6]:
!git clone https://github.com/n00bmaster68/ApplyingSODinAutoLabelling

Cloning into 'ApplyingSODinAutoLabelling'...
remote: Enumerating objects: 2101, done.
remote: Counting objects: 100% (2101/2101), done.
remote: Compressing objects: 100% (2080/2080), done.
remote: Total 2101 (delta 21), reused 2084 (delta 8), pack-reused 0
Receiving objects: 100% (2101/2101), 75.19 MiB | 60.96 MiB/s, done.
Resolving deltas: 100% (21/21), done.


In [2]:
#check whether device has GPU
# import tensorflow as tf
# tf.test.gpu_device_name()
# !nvidia-smi -L
!pip install torch
!pip install scikit-image
!pip install torchvision

import torch
print(torch.cuda.get_device_name(torch.cuda.current_device()))

  Using cached torch-1.8.1-cp37-cp37m-manylinux1_x86_64.whl (804.1 MB)
  Using cached scikit_image-0.18.1-cp37-cp37m-manylinux1_x86_64.whl (29.2 MB)
     |████████████████████████████████| 169 kB 4.1 MB/s eta 0:00:01
  Using cached imageio-2.9.0-py3-none-any.whl (3.3 MB)
  Using cached PyWavelets-1.1.1-cp37-cp37m-manylinux1_x86_64.whl (4.4 MB)
  Using cached torchvision-0.9.1-cp37-cp37m-manylinux1_x86_64.whl (17.4 MB)
Tesla T4


In [3]:
import torch
import torchvision
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import torch.optim as optim
import torchvision.transforms as standard_transforms

import numpy as np
import glob

from ApplyingSODinAutoLabelling.data_loader import Rescale
from ApplyingSODinAutoLabelling.data_loader import RescaleT
from ApplyingSODinAutoLabelling.data_loader import RandomCrop
from ApplyingSODinAutoLabelling.data_loader import CenterCrop
from ApplyingSODinAutoLabelling.data_loader import ToTensor
from ApplyingSODinAutoLabelling.data_loader import ToTensorLab
from ApplyingSODinAutoLabelling.data_loader import SalObjDataset

from ApplyingSODinAutoLabelling.model import BASNet

import ApplyingSODinAutoLabelling.deletable.pytorch_ssim as pytorch_ssim
import ApplyingSODinAutoLabelling.deletable.pytorch_iou as pytorch_iou

In [4]:
# ------- 1. define loss function --------

bce_loss = nn.BCELoss(size_average=True)
ssim_loss = pytorch_ssim.SSIM(window_size=11,size_average=True)
iou_loss = pytorch_iou.IOU(size_average=True)

def bce_ssim_loss(pred,target):

	bce_out = bce_loss(pred,target)
	ssim_out = 1 - ssim_loss(pred,target)
	iou_out = iou_loss(pred,target)

	loss = bce_out + ssim_out + iou_out
	return loss, ssim_out

def muti_bce_loss_fusion(d0, d1, d2, d3, d4, d5, d6, d7, labels_v):
  loss0, ssim0 = bce_ssim_loss(d0,labels_v)
  loss1, ssim1 = bce_ssim_loss(d1,labels_v)
  loss2, ssim2 = bce_ssim_loss(d2,labels_v)
  loss3, ssim3 = bce_ssim_loss(d3,labels_v)
  loss4, ssim4 = bce_ssim_loss(d4,labels_v)
  loss5, ssim5 = bce_ssim_loss(d5,labels_v)
  loss6, ssim6 = bce_ssim_loss(d6,labels_v)
  loss7, ssim7 = bce_ssim_loss(d7,labels_v)

  acc = (ssim0 + ssim1 + ssim2 + ssim3 + ssim4 + ssim5 + ssim6 + ssim7) / 8
  
  loss = loss0 + loss1 + loss2 + loss3 + loss4 + loss5 + loss6 + loss7
  print("l0: %3f, l1: %3f, l2: %3f, l3: %3f, l4: %3f, l5: %3f, l6: %3f\n"%(loss0.data,loss1.data,loss2.data,loss3.data,loss4.data,loss5.data,loss6.data))
  
  return loss0, loss, 1 - acc

/srv/conda/envs/saturn/lib/python3.7/site-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))


In [6]:
# ------- 2. set the directory of training dataset --------

data_dir = 'ApplyingSODinAutoLabelling/train_data/'
tra_image_dir = 'DUTS/DUTS-TR/DUTS-TR/im_aug/'
tra_label_dir = 'DUTS/DUTS-TR/DUTS-TR/gt_aug/'

image_ext = '.jpg'
label_ext = '.png'

model_dir = "ApplyingSODinAutoLabelling/saved_models/basnet_bsi/"


epoch_num = 400
batch_size_train = 8 
batch_size_val = 1
train_num = 0
val_num = 0

tra_img_name_list = glob.glob(data_dir + tra_image_dir + '*' + image_ext)

tra_lbl_name_list = []
for img_path in tra_img_name_list:
	img_name = img_path.split("/")[-1]

	aaa = img_name.split(".")
	bbb = aaa[0:-1]
	imidx = bbb[0]
	for i in range(1,len(bbb)):
		imidx = imidx + "." + bbb[i]

	tra_lbl_name_list.append(data_dir + tra_label_dir + imidx + label_ext)

print("---")
print("train images: ", len(tra_img_name_list))
print("train labels: ", len(tra_lbl_name_list))
print("---")

train_num = len(tra_img_name_list)

salobj_dataset = SalObjDataset(
    img_name_list=tra_img_name_list,
    lbl_name_list=tra_lbl_name_list,
    transform=transforms.Compose([
        RescaleT(256),
        RandomCrop(224),
        ToTensorLab(flag=0)]))
salobj_dataloader = DataLoader(salobj_dataset, batch_size=batch_size_train, shuffle=True, num_workers=1)


---
train images:  1000
train labels:  1000
---


In [7]:
# ------- 3. define model --------
# define the net
net = BASNet(3, 1)
# net.load_state_dict(torch.load("ApplyingSODinAutoLabelling/saved_models/basnet_bsi/basnet_bsi_itr_42000_train_1.170107_tar_0.025873.pth"))
if torch.cuda.is_available():
    net.cuda()

In [8]:
# ------- 4. define optimizer --------
print("---define optimizer...")
optimizer = optim.Adam(net.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0)


---define optimizer...


In [9]:
# ------- 5. training process --------
print("---start training...")
ite_num = 43626
running_loss = 0.0
running_tar_loss = 0.0
ite_num4val = 0
top_100_acc = []


for epoch in range(0, epoch_num):
    net.train()

    for i, data in enumerate(salobj_dataloader):
        ite_num = ite_num + 1
        ite_num4val = ite_num4val + 1

        inputs, labels = data['image'], data['label']

        inputs = inputs.type(torch.FloatTensor)
        labels = labels.type(torch.FloatTensor)

        # wrap them in Variable
        if torch.cuda.is_available():
            inputs_v, labels_v = Variable(inputs.cuda(), requires_grad=False), Variable(labels.cuda(),
                                                                                        requires_grad=False)
        else:
            inputs_v, labels_v = Variable(inputs, requires_grad=False), Variable(labels, requires_grad=False)

        # y zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        d0, d1, d2, d3, d4, d5, d6, d7 = net(inputs_v)
        loss2, loss, acc = muti_bce_loss_fusion(d0, d1, d2, d3, d4, d5, d6, d7, labels_v)

        loss.backward()
        optimizer.step()

        # # print statistics
        running_loss += loss.data
        running_tar_loss += loss2.data

        # del temporary outputs and loss
        del d0, d1, d2, d3, d4, d5, d6, d7, loss2, loss

        # print("ite_num4val: ", ite_num4val)

        print(f"[epoch: {epoch + 1}/{epoch_num}, batch: {(i + 1) * batch_size_train}/{train_num}, ite: {ite_num}] train loss: {format(running_loss / ite_num4val, '.4f')}, accuracy: {format(acc * 100, '.4f')}%, tar: {format(running_tar_loss / ite_num4val, '.4f')} ")

        if ite_num % 2000 == 0:  # save model every 2000 iterations

            torch.save(net.state_dict(), model_dir + "basnet_bsi_itr_%d_train_%3f_tar_%3f.pth" % (ite_num, running_loss / ite_num4val, running_tar_loss / ite_num4val))
            running_loss = 0.0
            running_tar_loss = 0.0
            net.train()  # resume train
            ite_num4val = 0
        
        if epoch + 1 >= 400:
            top_100_acc.append(acc * 100)

print('-------------Congratulations! Training Done!!!-------------')


---start training...


/srv/conda/envs/saturn/lib/python3.7/site-packages/torch/nn/functional.py:3458: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode)
/srv/conda/envs/saturn/lib/python3.7/site-packages/torch/nn/functional.py:1709: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


l0: 0.020569, l1: 0.021451, l2: 0.027077, l3: 0.040666, l4: 0.091220, l5: 0.197166, l6: 0.429669

[epoch: 327/400, batch: 8/1000, ite: 43627] train loss: 1.2589, accuracy: 94.4221%, tar: 0.0206 
l0: 0.024627, l1: 0.026028, l2: 0.033344, l3: 0.049147, l4: 0.097491, l5: 0.181382, l6: 0.427431

[epoch: 327/400, batch: 16/1000, ite: 43628] train loss: 1.2676, accuracy: 94.6877%, tar: 0.0226 
l0: 0.021815, l1: 0.023188, l2: 0.030266, l3: 0.042640, l4: 0.072216, l5: 0.149280, l6: 0.316113

[epoch: 327/400, batch: 24/1000, ite: 43629] train loss: 1.1692, accuracy: 95.8737%, tar: 0.0223 
l0: 0.019923, l1: 0.021978, l2: 0.029356, l3: 0.044496, l4: 0.073477, l5: 0.130958, l6: 0.272593

[epoch: 327/400, batch: 32/1000, ite: 43630] train loss: 1.0930, accuracy: 96.2943%, tar: 0.0217 
l0: 0.020918, l1: 0.022505, l2: 0.030285, l3: 0.043802, l4: 0.084441, l5: 0.187063, l6: 0.311302

[epoch: 327/400, batch: 40/1000, ite: 43631] train loss: 1.0771, accuracy: 96.2786%, tar: 0.0216 
l0: 0.023127, l1: 0.0